# Building on the Mark-Recapture model

### Changes vs previous version:
* All constants moved from global variables to a class, as class properties
* The model is made into a function
* The "running dotses" visualization is made conditional (so we haven't deleted it completely), but it is now inactivated by default

In [3]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
class Const:
    def __init__(self):
        self.N = 10
        self.nsteps = 100
        self.vmin = 0.01
        self.vmax = 0.05
        self.trap_size = 0.4
        self.trap_location = 'corner'
        self.death_rate = 0.01

In [5]:
def model(c, show_dots=False, ntrials=1):
    history = np.zeros((c.nsteps, ntrials))
    
    for itrial in range (ntrials):
        x = np.random.uniform(size=c.N)
        y = np.random.uniform(size=c.N)
        vx = np.random.uniform(size=c.N, low=c.vmin, high=c.vmax)*(
            2*np.random.randint(size=c.N,low=0,high=2) - 1)
        vy = np.random.uniform(size=c.N, low=c.vmin, high=c.vmax)*(
            2*np.random.randint(size=c.N,low=0,high=2) - 1)
        
        if c.trap_location == 'corner':
            color = np.where(np.logical_and(x<c.trap_size, y<c.trap_size), 1, 0)
        else:
            
        